# Lecture 24: Deep Learning


Applied Machine Learning


Professor: __Ramesh Natarajan__ \
Email: **ramesh.natarajan@yu.edu** \
Department of Computer Science \
Stern College and Yeshiva University

Date: Feb xx, 2024



## Acknowledgements

1. The starting point for developing the notebook-based instruction material was the set of notebooks provided by the previous  course instructor__[Prof. Zach Glassman](https://www.linkedin.com/in/zachary-glassman-7088844b/)__.
2. These notebooks have been augmented by similar instruction material taken from many, many other sources, including official documentation from many ML libraries and frameworks. I have adapted and modified the content where needed to make it compatible with student level, orientation and proficiency in this course.
3. The primary addition sources include:
   1. Course content from V. Kuleshov, Cornell Tech __[repo](https://github.com/kuleshov/cornell-cs5785-2020-applied-ml/blob/main/notebooks/lecture1-introduction.ipynb)__.
   2. Book and support material from Hal Daume II, __[A course in Machine Learning](http://ciml.info)__. __[repo](https://github.com/hal3/ciml/)__
   3. Book and support material from A. Geron, __[Hands-On Machine Learning with Scikit-Learn, Keras and Tensorflow (3rd edition)](https://homl.info/er3)__. __[repo](https://github.com/ageron/handson-ml3/blob/main/README.md)__




# Announcements

1. Problem Set 2 to be closed this week. Problem Set 3 still open for submission.
2. Recitation will review Deep Learning.
3. You should continue  to review and refresh yourself on the mathematical prerequisites for applied machine learning from part 1 for Deisenroth, Faisal and Ong, __[Mathematics for Machine Learning](https://mml-book.github.io/book/mml-book.pdf)__.



# Deep Learning  


<table>
    <td width="40%"><center>
        <img src="https://github.com/rameshnatarajanus/katz-aim5004-fall2025/blob/main/notebooks/img/Lecture-24/deep-learning-creative.png?raw=1" />      
    </center>
    Generated by Chat-GPT
     </td>  
    <td>
<font size = "+1.0">
Deep learning models are Multilayer Neural Networks which have led to especially significant improvements in image, video, speech and text modeling applications.  These improvements have been achieved through a combination of interesting algorithms (e.g. dropout regularization and stochastic gradient descent), rapid prototyping using several important frameworks that support automatic differentiation (e.g. Pytorch, Tensorflow, Jax),  high-performance hardware implementations on GPUs (for fast convolutions and matrix vector operations).  

</td></table>


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Training Neural Networks

This notebook gives a brief introduction to training Neural Networks and some of the pitfalls and problems you will see along the way.

It strives to give a basic overview of many topics, each one could easily be a lecture on its own.

## Gradient Descent

Lets dive into a bit more detail on the gradient descent algorithm..

In [ ]:
import tensorflow as tf
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    *make_blobs(n_samples=1000, centers=3, n_features=2, cluster_std=2, random_state=2)
)

An *epoch* is one pass over the training data, and a back-propagation computation to adjust the model weights is carried out for each example in the training data.  

For large datasets, this is too slow and inefficient, and therefore we consider using a batch of input examples for each back propagation step, using a  `batch_size` parameter .  In addition batching the data in this way is also very helpful for optimizing the use of parallel hardware such as GPUs.


In [ ]:
import time
class TimingCallback(tf.keras.callbacks.Callback):
    def __init__(self, times, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.times = times
        self._start_time = None

    def on_epoch_begin(self, epoch, logs=None):
        self._start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self._start_time)

In [ ]:
from tqdm.keras import TqdmCallback

In [ ]:
def train(batch_size, epochs=200):
    print(f'fitting for batch_size: {batch_size}')
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(2,)),
        tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.Dense(3)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.005),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    times = []
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0,
                        callbacks=[
                            TimingCallback(times),
                            TqdmCallback(verbose=1)
                        ],
                        )
    return history, times

In [ ]:
#batch_sizes = [1, 50, len(X_train)]
batch_sizes = [16, 32]

In [ ]:
# note, this will take several minutes to run....
results = [train(i) for i in batch_sizes]

In [ ]:
for (history, _), batch_size in zip(results, batch_sizes):
    plt.plot(history.history['accuracy'], label=batch_size)
plt.legend()
plt.title('accuracy score')
plt.xlabel('epoch')

In [ ]:
for (_, times), batch_size in zip(results, batch_sizes):
    plt.plot(times, '.-', label=batch_size)
plt.legend()
plt.title('times')
plt.xlabel('epoch')
plt.ylabel('seconds')

Lets talk about the three cases

- `batch_size=1` : online gradient descent
- `batch_size=N`: gradient descent
- `bach_size> 1 AND < N`: minibatch

### Stochastic gradient Descent

- In this case we are doing a complete forward and back propagation through the network on each step
- This is very slow
- Introduces lots of randomness into the network (which is not entirely bad).
- Only requires one sample at a time, so it can scale very well to big datasets

### Batch Gradient Descent

- In this case we look at all samples in the training set and move due to their average gradient.
- This can be faster as we need less steps in our optimization
- Can be get caught in local minima easily
- Requires all samples

### Minibatch Gradient Descent

- Combination of previous two
- Batch size is not 1, but more like 32
- Has advantages of being faster with fewer updates
- Has enough randomness to not be as prone to local minima
- Only requires a few samples, so can typically scale well

## Data Preprocessing

Algorithms can also be sensitive to input feature scaling and transformations.  For example, lets look at if we can do better in our model by rescaling the pixel values.

We will work with the fashion MNIST data from last lecture.

In [ ]:
_data = tf.keras.datasets.fashion_mnist.load_data()
(train_images, train_labels), (test_images, test_labels) = _data

def compile_and_fit(model):
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    epochs=10,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
        )
    return model

Lets start with a simple model using no rescaling.  We will be able to get about 80% validation error (note for a better test we could also use a hold out set.)

In [ ]:
from tensorflow.keras.utils import model_to_dot
from IPython.display import Image, display


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(140, activation='relu'),
    tf.keras.layers.Dense(10)
])


model_graph = model_to_dot(model, show_shapes=True, show_layer_names=True)
display(Image(model_graph.create_png()))
compile_and_fit(model)

Now we can add a `Rescaling` layer which will normalize all the inputs between 0 and 1 by dividing by the maximum value (255).

In [ ]:
train_images.max(), train_images.min()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Dense(140, activation='relu'),
    tf.keras.layers.Dense(10)
])

compile_and_fit(model)

## Data Augmentation and Jittering

We can see we do much better.  We can also introduce other augmentation techniques like shifting the data, rotating the data.  Here we will randomly rotate the image by a little bit.  

*Note:* These layers will be active only during training, but not during prediction.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(140, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    epochs=10,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
                    )

The augmentation is useful because it helps our model generalize to more possibilities.  It can even make the network resilient to things like rotation which often crop up in real world problems.

# Vanishing gradients

We know that the training requires understanding the computation of the  gradient using back propagation (via the chain rule of differentiation), which requires taking the product of the derivatives of the activation function that is used in each hidden layer.

As can be seen from the figure below for the case where the activation function is the hyperbolic tangent, the derivative of this function is zero except for small region of activation (between $\pm 2$).

This is known as the **vanishing gradient problem** since the network weights will not be updated in this case.  

In [ ]:
import numpy as np
x = np.linspace(-5, 5, 100)
plt.figure(figsize=(4,3))
plt.plot(x, np.tanh(x), label='tanh')
plt.plot(x, 1/(np.cosh(x)**2), label='d/dx(tanh)')
plt.legend()
plt.show()

### Activation Functions

In order to avoid the vanishing gradients problem, the activation function have been proposed


- ReLU Rectified Linear Unit
- Leaky ReLU
- Exponential Linear Unit
- Scaled Exponential Linear Unit

The ReLU is a very common choice since it avoids the vanishing gradients problems, since the gradient (at least for positive inputs) is always 1.

While it helps induce sparsity in the network due to its one-sidedness, in some cases this can also lead to segments of the neural network having vanishing gradient.  An alternative in this case is the  Leaky RelU

In [ ]:

from tensorflow.keras import activations

x = np.linspace(-3, 3, 100)
plt.figure(figsize=(4,3))
for func in ['relu',  'selu', 'elu', 'leaky_relu']:
    plt.plot(x, getattr(tf.keras.activations, func)(x), label=func)
plt.legend()
plt.show()

## Exploding gradients

Exploding gradient problem - While we have solved the vanishing gradient problem, sometimes large unbounded gradients can cause the gradient to get too large and blow up.  Often can be solved by proper initialization.

Takeaways - Lots of intricacies to training, need to keep track of proper initialization, activation functions etc.

# Optimizers

Many optimizers have been developed for Neural Networks that address some of the issues with the use of gradient descent.

For example, in SGD we have a `step_size` hyperparameter which controls the learning rate.  However this quantity can be hard to set without some experimentation - too large and it is hard to converge, too small and it converges but very slowly.


Lets look at a simple case with a few different learning rates.

In [ ]:
def loss(x):
    return .5 * x ** 2
def optimize(start_value, step_size, steps=20):
    vals = [start_value]
    for step in range(steps):
        val = vals[-1]
        vals.append(val - step_size * val)
    return np.array(vals)

fig, ax = plt.subplots(2, 1)
x = np.linspace(-5, 5)
ax[0].plot(x, loss(x))
colors = ['r', 'g', 'orange']
for step_size, c in zip([.01, .5, 1.7], colors):
    opt = optimize(5, step_size)
    ax[0].plot(opt, loss(opt), '.-', label=f'step_size={step_size}', c=c)
    ax[1].plot(opt, '.-', c=c)
ax[0].legend()
ax[1].set_xlabel('step')
ax[1].set_ylabel('val')
plt.show()

In order to improve this, there are a number of methods which help by various methods such as

momentum - (automatic) modify  gradients based on the values of previous gradients
adaptive learning rates - (user specified) manually control learning rates based on so-called learning rate curves

We can see how some of these work in practice by using some of the Tensorflow optimizers

In [ ]:
def run(optimizer, start_value, steps=100):
    var = tf.Variable(start_value)
    loss = lambda: .5 * var**2

    vals = [start_value]
    for step in range(steps):
        with tf.GradientTape() as g:
            g.watch(var)
            t = loss()
            gradients = g.gradient(t, [var])
        optimizer.apply_gradients(zip(gradients, [var]))
        vals.append(var.numpy())
    return vals
  # Calculate the gradients

LR = .05
for opt in [
    tf.keras.optimizers.SGD(learning_rate=LR),
    tf.keras.optimizers.RMSprop(learning_rate=LR),
    tf.keras.optimizers.Adam(learning_rate=LR)
]:
    vals = run(opt, 2.0)
    plt.plot(vals, '.-', label=opt.__class__.__name__)
plt.legend()
plt.xlabel('step')
plt.ylabel('var')
plt.show()

# Dropout

We have many parameters in deep learning models,  If you ever want to check, look at the model summary (need to compile it first)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Dense(140, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()

Even simple networks will have hundreds of thousands of parameters, so how to avoid overfitting?

The simplest method, and on that is perhaps surprisingly successful is the notion of dropout (although hopefully it won't be so surprising why after this lecture :))

The basic idea in dropout is the following

1. On each sample, randomly drop each weight with probability $p$
2. Rescale all weights to account for dropped weights
3. Proceed as normal

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(train_images, train_labels,
            validation_data=(test_images, test_labels),
            epochs=100,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
            )

### Why does this work?

A smaller network is less prone to overfitting

Each individual iteration is training a smaller network (kind of a like a Random Forest!).

Another way to think about it, is that each individual neuron cannot become too specialized as its only there part of the time.

# Checkpointing and Persisting Models

Once we train models, we need to persist them so that we can load them later to make predictions.  To do this is actually relatively simple as long as your entire "pipeline" is in a single model.

**Note:** not working in JupyterHub

In [ ]:
model.save('./saved_model.keras')

In [ ]:
from pathlib import Path
list(Path('./saved_model.keras').glob("*"))

In [ ]:
list(Path('./saved_model.keras/variables').glob("*"))

Tensorflow is saving a few things:

1. The model architecture (in the .pb file)
2. The weights - saved in the variables directory

We can load it as well.

In [ ]:
saved_model = tf.keras.models.load_model('./saved_model.keras')
saved_model.layers

Often we might train a model on one machine which has lots of compute and then run inference on a different machine, say a web service.  This persistence allows us to do this quite easily.

# Transfer Learning

One of the most important innovations in ML (at least in a practical sense) is the idea of Transfer Learning

Transfer learning allows us to train models on a large amount of general data and then reuse those models to train specialized models.

Intuitively, we can think of these pre-trained models as really awesome feature extractors which we can then use to train smaller neural networks (or tune them).

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(train_images[25])
plt.show()

We are just focusing on the training here, however, it would be a good idea to leave out some of the dataset for final testing to make sure that as we tune hyperparameters we are not including any knowledge we shouldn't be including.

Lets start by just training our previous model on this dataset.

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Dense(140, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels), epochs=10)

We see this model didn't do so well.  The reason is that this model does not have the requisite complexity to capture the details of this dataset.

On the other hand, this dataset is not super large, can we train a complex model?

Luckily we don't have to.  We can use a pre-trained model which is trained on large amounts of relatively similiar data, then simply train a small model on top to encode our particular problem.

The intuitive idea is that these large models are learning lots of fundamental features about images, we can use them as feature extractors.

Keras makes this quite easy, it comes with a bunch of pre-built 'applications' for things like image process as well as NLP.  We will use the `MobileNetV3Small` for demonstration purposes here.

First we need to create the model, notice the model is set as `trainable=False`, this is because we do not at this point want to train the weights of this model, only use them.

In [ ]:
def get_base_model():
    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    return base_model

Now we can create a function `get_model` which will create our overall model.  We will use an alternate way of create models in Keras, sometimes called the *functional* API.  It will make it a bit easier to use our pre-trained model.  Note the following

- We rescale the images, models often will have an expected feature input.
- We apply a `GlobalAveragePooling2D` layer.  This averages over the feature vector so we can then apply our final classification layers to it.

In [ ]:
def get_model():
    base_model = get_base_model()
    inputs = tf.keras.Input(shape=(32, 32, 3))
    scale =  tf.keras.layers.Lambda(
        lambda image : tf.image.resize_with_pad(image, 224, 224)
    )(inputs)
    rescale = tf.keras.applications.mobilenet_v3.preprocess_input(scale)
    x = base_model(rescale, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(.3)(x)
    outputs = tf.keras.layers.Dense(10)(x)
    return tf.keras.Model(inputs, outputs)

In [ ]:
get_model()

We will use a strategy here.  Typically we do not need to do this, however, if we want to make use of a TPU or GPU, this will make the code translate more easily.  

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = get_model()
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    model.summary()

Note that this next cell will run really slowly if we only have CPU installed, so we will only run for a little bit, however, we can switch to a GPU or TPU with almost the same code and see a very large performance gain.

In [ ]:
history = model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    epochs=1
)

If we want to squeeze a bit more performance out, we can actually retrain part of the original network, however, we want to make sure we are not retraining from start.

In [ ]:
model.layers

In [ ]:
base_model = model.layers[2]

In [ ]:
base_model.trainable

We will make the base model trainable, but only the final few layers.

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:200]:
    layer.trainable = False

In [ ]:
with strategy.scope():
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer = tf.keras.optimizers.RMSprop(learning_rate=.00001),
                  metrics=['accuracy'])

In [ ]:
model.summary()

Now we can continue training the model.

In [ ]:
with strategy.scope():
    history_final = model.fit(
        train_images,
        train_labels,
        validation_data=(test_images, test_labels),
        initial_epoch=history.epoch[-1],
        epochs=2
    )